# Index

- **Python Libraries**
- **Task 2**
    - *Reading Preprocessed Datasets*
    - *Exercise 1*
    - *Exercise 2*
        - *Balancing Strategy Definition*
        - *Exercise 2.1*
        - *Exercise 2.2*
        - *Exercise 2.3*
    - *Exercise 3 - Concentration*
    - *Writing Balanced Datasets*


# Pyhton Libraries

In [1]:
import torch
import pandas as pd
import numpy as np
import datetime as dt
import random

# Task 2

## Reading Preprocessed Datasets

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
read_data_path  = "../content/drive/MyDrive/ML/preprocessed"

ap   = pd.read_csv(read_data_path+'/anagraficapazientiattivi.csv', header=0 ,names=['idcentro','idana','sesso','annodiagnosidiabete','tipodiabete','annonascita','annoprimoaccesso','annodecesso','label'])
diag = pd.read_csv(read_data_path+'/diagnosi.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
elp  = pd.read_csv(read_data_path+'/esamilaboratorioparametri.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
ei   = pd.read_csv(read_data_path+'/esamistrumentali.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pdf  = pd.read_csv(read_data_path+'/prescrizionidiabetefarmaci.csv', header=0 ,names=['idcentro','idana','data','codiceatc','quantita','idpasto','descrizionefarmaco'])
pdnf = pd.read_csv(read_data_path+'/prescrizionidiabetenonfarmaci.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pnd  = pd.read_csv(read_data_path+'/prescrizioninondiabete.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])

In [4]:
print(ap.shape)
ap.head()

(2242, 9)


,idcentro,idana,sesso,annodiagnosidiabete,tipodiabete,annonascita,annoprimoaccesso,annodecesso,label
0,1,5,M,1986.0,5,1942,1990.0,2014.0,1
1,1,46,F,1982.0,5,1937,1997.0,2016.0,1
2,1,60,F,1987.0,5,1923,1987.0,2010.0,0
3,1,65,M,1997.0,5,1942,1997.0,2018.0,1
4,1,74,M,1995.0,5,1930,1996.0,2013.0,0


In [5]:
print(diag.shape)
diag.head()

(155217, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,1997-12-01,AMD247,410
1,1,5,2000-06-01,AMD247,434.01
2,1,5,2000-06-01,AMD247,434.91
3,1,5,2000-12-02,AMD049,S
4,1,5,2000-12-02,AMD247,36.10


In [6]:
print(elp.shape)
elp.head()

(451271, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2005-01-18,AMD001,169.000000
1,1,5,2005-01-18,AMD002,76.000000
2,1,5,2005-01-18,AMD004,90.232558
3,1,5,2005-01-18,AMD005,47.346939
4,1,5,2005-06-06,AMD007,138.225058


In [7]:
print(ei.shape)
ei.head()

(21652, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2006-01-04,AMD051,N
1,1,5,2006-11-14,AMD041,P
2,1,5,2006-11-20,AMD040,P
3,1,5,2007-06-01,AMD040,N
4,1,5,2008-01-16,AMD040,N


In [8]:
print(pdf.shape)
pdf.head()

(134030, 7)


,idcentro,idana,data,codiceatc,quantita,idpasto,descrizionefarmaco
0,1,5,2005-01-18,A10BA02,2.0,1.0,METFORAL*50CPR RIV 500mg
1,1,5,2005-01-18,A10BA02,2.0,3.0,METFORAL*50CPR RIV 500mg
2,1,5,2005-01-18,A10BA02,2.0,5.0,METFORAL*50CPR RIV 500mg
3,1,5,2005-01-18,A10BB12,1.0,3.0,AMARYL*30CPR 2mg
4,1,5,2005-06-21,A10BA02,2.0,1.0,METFORAL*50CPR RIV 500mg


In [9]:
print(pdnf.shape)
pdnf.head()

(14071, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2008-06-20,AMD152,NaN
1,1,5,2013-08-27,AMD152,NaN
2,1,5,2013-12-31,AMD086,S
3,1,5,2013-12-31,AMD228,S
4,1,46,2007-01-26,AMD152,4015630065


In [10]:
print(pnd.shape)
pnd.head()

(147973, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2005-01-18,AMD121,C09AA05
1,1,5,2005-01-18,AMD124,C10AA05
2,1,5,2005-01-18,AMD124,C10AX06
3,1,5,2005-06-21,AMD121,C09AA05
4,1,5,2005-06-21,AMD124,C10AA05


## Exercise 1

*Class imbalance #1* - not all patients will have a cardiovascular event within the stabilised six-month period. Thus, we would expect that the class distribution is highly imbalanced.

- For each patient $p_i$ such that $y(p_i)=0$, eliminate the last six months of history to avoid giving the model prediction hints into the future.
- For each patient $p_i$ such that $y(p_i)=1$, create $m$ copies $\lbrace p_i^1, \cdots, p_i^m \rbrace$ such that all the cardiovascular events in the last six months are eliminated, and the other events are shuffled and cancelled at random.

In this way, you have a sort of balancing criterion (i.e., up-sampling the minority class).

First we divide the **ap** dataframe into:
- **ap_class0** each patient $p_i$ has $y(p_i)=0$
- **ap_class1** each patient $p_i$ has $y(p_i)=1$

In [11]:
print(f"ap has size        => {ap.shape}")
ap_class0 = ap[ap.label == 0]
print(f"ap_class0 has size => {ap_class0.shape}")
ap_class1 = ap[ap.label == 1]
print(f"ap_class1 has size => {ap_class1.shape}")


ap has size        => (2242, 9)
ap_class0 has size => (1221, 9)
ap_class1 has size => (1021, 9)


And now the rest of the dataframes are also divided into **df_class0** and **df_class1** following the same criteria:

In [12]:
diag_class0 = pd.merge(diag, ap_class0, on=['idcentro','idana'])[diag.columns]
elp_class0  = pd.merge(elp, ap_class0, on=['idcentro','idana'])[elp.columns]
ei_class0   = pd.merge(ei, ap_class0, on=['idcentro','idana'])[ei.columns]
pdf_class0  = pd.merge(pdf, ap_class0, on=['idcentro','idana'])[pdf.columns]
pdnf_class0 = pd.merge(pdnf, ap_class0, on=['idcentro','idana'])[pdnf.columns]
pnd_class0  = pd.merge(pnd, ap_class0, on=['idcentro','idana'])[pnd.columns]

diag_class1 = pd.merge(diag, ap_class1, on=['idcentro','idana'])[diag.columns]
elp_class1  = pd.merge(elp, ap_class1, on=['idcentro','idana'])[elp.columns]
ei_class1   = pd.merge(ei, ap_class1, on=['idcentro','idana'])[ei.columns]
pdf_class1  = pd.merge(pdf, ap_class1, on=['idcentro','idana'])[pdf.columns]
pdnf_class1 = pd.merge(pdnf, ap_class1, on=['idcentro','idana'])[pdnf.columns]
pnd_class1  = pd.merge(pnd, ap_class1, on=['idcentro','idana'])[pnd.columns]

We define a function that eliminates the events during the last 6 months for each patient

In [13]:
def dropLastSixMonths(df:pd.DataFrame, patients: pd.DataFrame) -> pd.DataFrame:

    newDF = pd.DataFrame(columns=df.columns)

    # We iterate for each patient on patients
    for p in patients.itertuples():
        # Take the cardiovascular events of the patient p
        aux = df[(df.idcentro==p.idcentro) & (df.idana==p.idana)]

        # Calculate the last valid data
        aux = aux.sort_values('data', ascending=False)
        lastData = next(aux.itertuples()).data
        lastData = dt.date.fromisoformat(lastData)
        limitData = lastData - dt.timedelta(days=30*6)

        # Take only the events before this limit data
        aux = aux[aux.data < limitData.strftime("%Y-%m-%d")]

        # We concatenate this with the new DataFrame
        newDF = pd.concat([newDF, aux])

    return newDF

Now we execute the function to every dataframe in our preprocessed dataset for the **ap_class0** patients:

In [14]:
aux = diag_class0.shape[0]
diag_class0 = dropLastSixMonths(diag_class0, ap_class0)
print(f"Diag_class0: {aux} => {diag_class0.shape[0]}")

aux = elp_class0.shape[0]
elp_class0 = dropLastSixMonths(elp_class0, ap_class0)
print(f"Elp_class0: {aux} => {elp_class0.shape[0]}")

aux = ei_class0.shape[0]
ei_class0 = dropLastSixMonths(ei_class0, ap_class0)
print(f"Ei_class0: {aux} => {ei_class0.shape[0]}")

aux = pdf_class0.shape[0]
pdf_class0 = dropLastSixMonths(pdf_class0, ap_class0)
print(f"Pdf_class0: {aux} => {pdf_class0.shape[0]}")

aux = pdnf_class0.shape[0]
pdnf_class0 = dropLastSixMonths(pdnf_class0, ap_class0)
print(f"Pdnf_class0: {aux} => {pdnf_class0.shape[0]}")

aux = pnd_class0.shape[0]
pnd_class0 = dropLastSixMonths(pnd_class0, ap_class0)
print(f"Pnd_class0: {aux} => {pnd_class0.shape[0]}")

Diag_class0: 68853 => 60598
Elp_class0: 234208 => 203837
Ei_class0: 10714 => 8519
Pdf_class0: 69059 => 61643
Pdnf_class0: 7983 => 6003
Pnd_class0: 71844 => 63004


Now for the **ap_class1** patients:

In [15]:
aux = diag_class1.shape[0]
diag_class1 = dropLastSixMonths(diag_class1, ap_class1)
print(f"Diag_class1: {aux} => {diag_class1.shape[0]}")

aux = elp_class1.shape[0]
elp_class1 = dropLastSixMonths(elp_class1, ap_class1)
print(f"Elp_class1: {aux} => {elp_class1.shape[0]}")

aux = ei_class1.shape[0]
ei_class1 = dropLastSixMonths(ei_class1, ap_class1)
print(f"Ei_class1: {aux} => {ei_class1.shape[0]}")

aux = pdf_class1.shape[0]
pdf_class1 = dropLastSixMonths(pdf_class1, ap_class1)
print(f"Pdf_class1: {aux} => {pdf_class1.shape[0]}")

aux = pdnf_class1.shape[0]
pdnf_class1 = dropLastSixMonths(pdnf_class1, ap_class1)
print(f"Pdnf_class1: {aux} => {pdnf_class1.shape[0]}")

aux = pnd_class1.shape[0]
pnd_class1 = dropLastSixMonths(pnd_class1, ap_class1)
print(f"Pnd_class1: {aux} => {pnd_class1.shape[0]}")

Diag_class1: 86364 => 73887
Elp_class1: 217063 => 189759
Ei_class1: 10938 => 8790
Pdf_class1: 64971 => 57872
Pdnf_class1: 6088 => 4478
Pnd_class1: 76129 => 67051


Now let's assign idcopy = 1 to every single patient on ap_class0

In [16]:
def insertIdCopyClass0(df:pd.DataFrame) -> pd.DataFrame:

    cols = list(df.columns)
    if not(np.isin('idcopy',cols)):
        cols.insert(2,'idcopy')
        df['idcopy'] = 1
        df = df[cols]
    return df

In [17]:
pd.options.mode.chained_assignment = None  # default='warn'

ap_class0   = insertIdCopyClass0(ap_class0)
diag_class0 = insertIdCopyClass0(diag_class0)
elp_class0  = insertIdCopyClass0(elp_class0)
ei_class0   = insertIdCopyClass0(ei_class0)
pdf_class0  = insertIdCopyClass0(pdf_class0)
pdnf_class0 = insertIdCopyClass0(pdnf_class0)
pnd_class0  = insertIdCopyClass0(pnd_class0)

diag_class0.head()
pd.options.mode.chained_assignment = 'warn'  # default='warn'

At this point, we are done with the **ap_class0** patients and their events, so we focus on the **ap_class1** ones. We now have to create $m$ (= 3 for example) copies of each patient $p_i$ such that $y(p_i)=1$, and we shuffle or eliminate cardiovascular events randomly just to *oversample* the dataset:

In [18]:
def newMCopies(patients:pd.DataFrame, copies:int) -> pd.DataFrame:

    cols = patients.columns.tolist()
    cols = cols[0:2]+['idcopy']+cols[2:]
    newPatients = pd.DataFrame(columns=cols)

    for p in patients.iterrows():
        for i in range(copies):
            newP = dict(p[1])
            newP['idcopy'] = i+1
            newPatients.loc[newPatients.shape[0]] = newP

    return newPatients

In [19]:
M = 3
ap_class1_copies = newMCopies(ap_class1, M)

We can check that every patient has been cloned M times, and the only identification for each clone $p_i^j$ is $i=\lbrace$ idcentro, idana $\rbrace$ and $j = \lbrace$ idcopy $\rbrace$

In [20]:
print("Copies of Patient 1:")
ap_class1_copies.iloc[0:1*M]

Copies of Patient 1:


,idcentro,idana,idcopy,sesso,annodiagnosidiabete,tipodiabete,annonascita,annoprimoaccesso,annodecesso,label
0,1,5,1,M,1986.0,5,1942,1990.0,2014.0,1
1,1,5,2,M,1986.0,5,1942,1990.0,2014.0,1
2,1,5,3,M,1986.0,5,1942,1990.0,2014.0,1


In [21]:
print("Copies of Patient 2:")
ap_class1_copies.iloc[M:2*M]

Copies of Patient 2:


,idcentro,idana,idcopy,sesso,annodiagnosidiabete,tipodiabete,annonascita,annoprimoaccesso,annodecesso,label
3,1,46,1,F,1982.0,5,1937,1997.0,2016.0,1
4,1,46,2,F,1982.0,5,1937,1997.0,2016.0,1
5,1,46,3,F,1982.0,5,1937,1997.0,2016.0,1


In [22]:
def newMCopiesRandom(df: pd.DataFrame, copies:int) -> pd.DataFrame:

  list_of_dataframes = []

  # We iterate over the patients with class = 1
  for i in ap_class1.itertuples():

    # We get the events of df associated with the patient p
    events_patient = df.loc[(df['idcentro'] == i.idcentro) & (df['idana'] == i.idana)]
    events_patient['idcopy']=0

    cols = list(df.columns)
    cols.insert(2,'idcopy')
    events_patient = events_patient[cols]

    for j in range(copies):

      # We create a new dataframe with the events of patient (p_i^j) and assign the idcopy to (j+1)
      events_patient['idcopy'] = j+1

      # Now we decide randomly if each copy of the patient p is going to have the events generating a boolean decision list
      # We also include a random parameter weight in order to become False choice more probable than True.
      weight = random.randint(50,70)/100
      choice = random.choices([False,True],weights=[weight,1-weight], k=events_patient.shape[0])

      # And add the new dataframe to the list of dataframes
      list_of_dataframes.append(events_patient[choice])

  newDf = pd.concat(list_of_dataframes, axis=0)
  return newDf.reset_index(drop=True)

In [23]:
pd.options.mode.chained_assignment = None  # default='warn'
diag_class1_copies = newMCopiesRandom(diag_class1, M)

In [24]:
print("Events of Patient 1 Copy 1:", diag_class1_copies.loc[(diag_class1_copies['idcentro']==1)&
                                                            (diag_class1_copies['idana']==5)&
                                                            (diag_class1_copies['idcopy']==1)].shape[0])
print("Events of Patient 1 Copy 2:", diag_class1_copies.loc[(diag_class1_copies['idcentro']==1)&
                                                            (diag_class1_copies['idana']==5)&
                                                            (diag_class1_copies['idcopy']==2)].shape[0])
print("Events of Patient 1 Copy 3:", diag_class1_copies.loc[(diag_class1_copies['idcentro']==1)&
                                                            (diag_class1_copies['idana']==5)&
                                                            (diag_class1_copies['idcopy']==3)].shape[0])

Events of Patient 1 Copy 1: 81
Events of Patient 1 Copy 2: 69
Events of Patient 1 Copy 3: 67


In [25]:
elp_class1_copies  = newMCopiesRandom(df=elp_class1, copies=M)


In [26]:
ei_class1_copies   = newMCopiesRandom(df=ei_class1, copies=M)

In [27]:
pdf_class1_copies  = newMCopiesRandom(df=pdf_class1, copies=M)


In [28]:
pdnf_class1_copies = newMCopiesRandom(df=pdnf_class1, copies=M)


In [29]:
pnd_class1_copies  = newMCopiesRandom(df=pnd_class1, copies=M)
pd.options.mode.chained_assignment = 'warn'

## Exercise 2

*Class Imbalance #2* - Action item *#1* isn't going to be sufficient for balancing purposes. Propose your balancing strategy - possibly an advanced approach - and evaluate:
- **vanilla-LTSM** (*Exercise 2.1*)
- **T-LTSM** (*Exercise 2.2*)
- **PubMedBERT** (*Exercise 2.3*)

on the balanced version of the dataset.

### Balancing Strategy Definition

We are going to keep only the events that are directly related to cardiovascular problems, this means for example:

- Cholesterol
- Diabetes
- Glycemia
- Smoking Patient
- Patient's Weight
- $\hspace{1cm}\vdots$

In [30]:
# We make a list with the events that we are going to take into account
codes = ['AMD001', 'AMD002', 'AMD003', 'AMD004', 'AMD005', 'AMD007', 'AMD010', 'AMD022', 'AMD034', 'AMD040', 'AMD042', 'AMD043', 'AMD044', 'AMD045', 'AMD046', 'AMD047', 'AMD048', 'AMD049', 'AMD055', 'AMD066', 'AMD069', 'AMD070', 'AMD071', 'AMD081', 'AMD082', 'AMD083', 'AMD086', 'AMD096', 'AMD097', 'AMD116', 'AMD118', 'AMD121', 'AMD132', 'AMD152', 'AMD204', 'AMD205', 'AMD208', 'AMD300', 'AMD303', 'AMD306', 'AMD307']

def selectEvents(df:pd.DataFrame, events_codes:list) -> pd.DataFrame:
    return df[df.codiceamd.isin(events_codes)]

# We select from the tables only the events with codes in the list
diag_cardEvents = selectEvents(diag,codes)

pnd_cardEvents = selectEvents(pnd,codes)

pdnf_cardEvents = selectEvents(pdnf,codes)

ei_cardEvents = selectEvents(ei,codes)

elp_cardEvents = selectEvents(elp,codes)

print(f"diag: {diag.shape[0]:6d} ==> {diag_cardEvents.shape[0]:6d}")
print(f"pnd:  {pnd.shape[0]:6d} ==> {pnd_cardEvents.shape[0]:6d}")
print(f"pdnf: {pdnf.shape[0]:6d} ==> {pdnf_cardEvents.shape[0]:6d}")
print(f"ei:   {ei.shape[0]:6d} ==> {ei_cardEvents.shape[0]:6d}")
print(f"elp:  {diag.shape[0]:6d} ==> {elp_cardEvents.shape[0]:6d}")

diag: 155217 ==>  40450
pnd:  147973 ==>  82036
pdnf:  14071 ==>  12729
ei:    21652 ==>   9475
elp:  155217 ==> 205641


In [31]:
print(f"Patients with class = 0: {ap_class0.shape[0]}")
print(f"Patients with class = 1: {ap_class1_copies.shape[0]}")

Patients with class = 0: 1221
Patients with class = 1: 3063


In [32]:
diag_aux = diag_cardEvents.filter(items=['idcentro', 'idana', 'codiceamd']).drop_duplicates()
diag_aux = diag_aux.filter(items=['idcentro', 'idana'])

pnd_aux = pnd_cardEvents.filter(items=['idcentro', 'idana', 'codiceamd']).drop_duplicates()
pnd_aux = pnd_aux.filter(items=['idcentro', 'idana'])

pdnf_aux = pdnf_cardEvents.filter(items=['idcentro', 'idana', 'codiceamd']).drop_duplicates()
pdnf_aux = pdnf_aux.filter(items=['idcentro', 'idana'])

ei_aux = ei_cardEvents.filter(items=['idcentro', 'idana', 'codiceamd']).drop_duplicates()
ei_aux = ei_aux.filter(items=['idcentro', 'idana'])

elp_aux = elp_cardEvents.filter(items=['idcentro', 'idana', 'codiceamd']).drop_duplicates()
elp_aux = elp_aux.filter(items=['idcentro', 'idana'])

suma= pd.concat(
    [diag_aux.groupby(['idcentro', 'idana']).value_counts(),
     pnd_aux.groupby(['idcentro', 'idana']).value_counts(),
     pdnf_aux.groupby(['idcentro', 'idana']).value_counts(),
     ei_aux.groupby(['idcentro', 'idana']).value_counts(),
     elp_aux.groupby(['idcentro', 'idana']).value_counts()],
    axis=1
    ).sum(axis=1)

In [33]:
# Here we have the patients with their total number of events
suma.sort_values()

idcentro  idana
203       4598      6.0
99        8027      7.0
1         11723     7.0
113       1796      7.0
1         2711      7.0
                   ... 
202       399      23.0
162       3332     23.0
164       43       23.0
162       1686     23.0
          1399     24.0
Length: 2242, dtype: float64

In [34]:
# Now we select those who have more than a certain threshold
threshold=17
suma=suma[suma>=threshold]
l=suma.index

In [35]:
# We turn our list into a dataframe and clean the patiens table
tupla1, tupla2 = [i for i, j in l], [j for i,j in l]

df1 = pd.DataFrame()
df1['idcentro']=tupla1
df1['idana']=tupla2
df1.head()

ap_aux  = pd.merge(ap_class1_copies, df1, on=['idcentro','idana'])
ap_aux.head()

,idcentro,idana,idcopy,sesso,annodiagnosidiabete,tipodiabete,annonascita,annoprimoaccesso,annodecesso,label
0,1,5,1,M,1986.0,5,1942,1990.0,2014.0,1
1,1,5,2,M,1986.0,5,1942,1990.0,2014.0,1
2,1,5,3,M,1986.0,5,1942,1990.0,2014.0,1
3,1,46,1,F,1982.0,5,1937,1997.0,2016.0,1
4,1,46,2,F,1982.0,5,1937,1997.0,2016.0,1


In [36]:
# We eliminate from the other tables the events of patients that have been just eliminated from the patients table
diag_aux = diag.merge(ap_aux, on=['idcentro', 'idana'])
pnd_aux = pnd.merge(ap_aux, on=['idcentro', 'idana'])
pdf_aux = pdf.merge(ap_aux, on=['idcentro', 'idana'])
pdnf_aux = pdnf.merge(ap_aux, on=['idcentro', 'idana'])
ei_aux = ei.merge(ap_aux, on=['idcentro', 'idana'])
elp_aux = elp.merge(ap_aux, on=['idcentro', 'idana'])

patientsOfInterest_labeled_aux = pd.concat([ap_class0,ap_aux]).filter(['idcentro','idana','idcopy','label'])

# Now we see that we have more patients with label 1 than patients with label 0
patientsOfInterest_labeled_aux['label'].value_counts()



0    1221
1    1068
Name: label, dtype: int64

### Exercise 2.1 - *Vanilla LSTM*

Let's try to use **cuda** (if possible) for a better performance:

In [37]:
from torch import cuda
DEVICE = 'cuda' if cuda.is_available() else 'cpu'
print("Using {torch.cuda.get_device_name(DEVICE)}")

Using {torch.cuda.get_device_name(DEVICE)}


We are just taking into a count the macro-events and the micro-events:

In [38]:
# First we concatenate all the dataframes, the ones with class = 0 with the others, with class = 1
ap_class   = pd.concat([ap_class0,ap_class1_copies],    axis=0)
diag_class = pd.concat([diag_class0,diag_class1_copies],axis=0)
elp_class  = pd.concat([elp_class0,elp_class1_copies],  axis=0)
ei_class   = pd.concat([ei_class0,ei_class1_copies],    axis=0)
pdf_class  = pd.concat([pdf_class0,pdf_class1_copies],  axis=0)
pdnf_class = pd.concat([pdnf_class0,pdnf_class1_copies],axis=0)
pnd_class  = pd.concat([pnd_class0,pnd_class1_copies],  axis=0)

### Writing Balanced Datasets

In [83]:
write_data_path = "/content/drive/MyDrive/ML/balanced"

ap_class.to_csv(write_data_path+"/anagraficapazientiattivi.csv", index=False, )
diag_class.to_csv(write_data_path+"/diagnosi.csv", index=False)
elp_class.to_csv(write_data_path+"/esamilaboratorioparametri.csv", index=False)
ei_class.to_csv(write_data_path+"/esamistrumentali.csv", index=False)
pdf_class.to_csv(write_data_path+"/prescrizionidiabetefarmaci.csv", index=False)
pdnf_class.to_csv(write_data_path+"/prescrizionidiabetenonfarmaci.csv", index=False)
pnd_class.to_csv(write_data_path+"/prescrizioninondiabete.csv", index=False)

In [40]:
ap_class.sort_values(by=['idcentro','idana']).head(5)

,idcentro,idana,idcopy,sesso,annodiagnosidiabete,tipodiabete,annonascita,annoprimoaccesso,annodecesso,label
0,1,5,1,M,1986.0,5,1942,1990.0,2014.0,1
1,1,5,2,M,1986.0,5,1942,1990.0,2014.0,1
2,1,5,3,M,1986.0,5,1942,1990.0,2014.0,1
3,1,46,1,F,1982.0,5,1937,1997.0,2016.0,1
4,1,46,2,F,1982.0,5,1937,1997.0,2016.0,1


Let's just take into a count the macro-events, so let's prepare the *diag_class* and the *ap_class* dataframe:

In [41]:

# At first, we merge with the patient data
features_lstm = pd.merge(diag_class, ap_class, on=['idcentro','idana','idcopy'])

# We create a single ID column that combines the other three:
features_lstm['id'] = features_lstm.apply(lambda x: f"{x['idcentro']}_{x['idana']}_{x['idcopy']}",axis=1)

# We reorder the columns
features_lstm = features_lstm[['id']+[x for x in features_lstm.columns if x!='id']]

# We drop the other ID_columns
features_lstm.drop(columns=['idcentro','idana','idcopy'], inplace=True)

# We categorize the columns that contain text
categorical_columns = ['codiceamd', 'valore', 'sesso']
for col in categorical_columns:
    features_lstm[col] = features_lstm[col].astype('category')
    features_lstm[col] = features_lstm[col].cat.codes

# We convert every columns into float type
numerical_columns = [col for col in features_lstm.columns if col not in ['id','data']]
for col in numerical_columns:
    features_lstm[col] = features_lstm[col].astype('float')

features_lstm.head(10)



<ipython-input-41-cd12b34edbcf>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_lstm.drop(columns=['idcentro','idana','idcopy'], inplace=True)
<ipython-input-41-cd12b34edbcf>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_lstm[col] = features_lstm[col].astype('category')
<ipython-input-41-cd12b34edbcf>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

,id,data,codiceamd,valore,sesso,annodiagnosidiabete,tipodiabete,annonascita,annoprimoaccesso,annodecesso,label
0,1_60_1,2009-07-16,46.0,550.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0
1,1_60_1,2009-04-29,6.0,253.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0
2,1_60_1,2009-04-29,43.0,267.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0
3,1_60_1,2008-12-03,32.0,550.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0
4,1_60_1,2008-12-03,43.0,196.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0
5,1_60_1,2008-12-03,43.0,193.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0
6,1_60_1,2008-11-10,46.0,550.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0
7,1_60_1,2007-07-29,46.0,550.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0
8,1_60_1,2007-02-02,7.0,170.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0
9,1_60_1,2007-02-02,43.0,267.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0


Now we build our model using *Keras*:

In [42]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.metrics import BinaryAccuracy

X_columns = [col for col in features_lstm if col not in ['id','label','data']]
y_columns = ['label']

Vanilla_LSTM = Sequential()
Vanilla_LSTM
Vanilla_LSTM.add(LSTM(100, activation='tanh', return_sequences=True, input_shape=(1, len(X_columns))))
Vanilla_LSTM.add(LSTM(49, activation='tanh'))
Vanilla_LSTM.add(Dense(1, activation='sigmoid'))
Vanilla_LSTM.compile(optimizer=Adam(learning_rate=1e-3),
              loss=BinaryCrossentropy(),
              metrics=[BinaryAccuracy()])

Now we create variable-size batches such that each batch contains only the events associated to the previous patients each step:



In [43]:
grouped_events = features_lstm.groupby(['id'])

for it, (ids, features) in enumerate(grouped_events):
    batch = features[features['id']==ids].sort_values(['data'])
    X = batch[X_columns]
    X = np.resize(X, (X.shape[0],1,X.shape[1]))
    y = batch[y_columns]
    if it % 200 == 0:
        print(f"Patient {it}/{len(ap_class)}")
    Vanilla_LSTM.fit(X,y,batch_size=len(X),epochs=10,verbose = 1 if it % 200 == 0 else 0)


<ipython-input-43-ed7914f99bd1>:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for it, (ids, features) in enumerate(grouped_events):


Patient 0/4284
Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 0.6981 - binary_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 17ms/step - loss: 0.6675 - binary_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 23ms/step - loss: 0.6409 - binary_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 18ms/step - loss: 0.6157 - binary_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 18ms/step - loss: 0.5909 - binary_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 16ms/step - loss: 0.5645 - binary_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 15ms/step - loss: 0.5369 - binary_accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 17ms/step - loss: 0.5138 - binary_accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 16ms/step - loss: 0.4909 - binary_accuracy: 1.0000
Epoch 10/10
1/1 [==============================

KeyboardInterrupt: ignored

In [ ]:
# We take a single batch to evaluate the model
rand_index = random.randint(0,len(ap_class))
rand_id = tuple(ap_class.iloc[rand_index, :3])
rand_id = f"{rand_id[0]}_{rand_id[1]}_{rand_id[2]}"
rand_batch = features_lstm[features_lstm['id']==rand_id]
print(rand_batch)

In [ ]:
X = rand_batch[X_columns]
X = np.resize(X, (X.shape[0],1,X.shape[1]))
Vanilla_LSTM.evaluate(x=X, y=rand_batch[y_columns])

### Exercise 2.2 - *T-LSTM*

In [44]:
import tensorflow.compat.v1 as tf

# disabling eager mode
tf.compat.v1.disable_eager_execution()


In [45]:
#We define the T-LSTM

class TLSTM(object):
    def init_weights(self, input_dim, output_dim, name, std=0.1, reg=None):
        return tf.get_variable(name,shape=[input_dim, output_dim],initializer=tf.random_normal_initializer(0.0, std),regularizer = reg)

    def init_bias(self, output_dim, name):
        return tf.get_variable(name,shape=[output_dim],initializer=tf.constant_initializer(1.0))

    def no_init_weights(self, input_dim, output_dim, name):
        return tf.get_variable(name,shape=[input_dim, output_dim])

    def no_init_bias(self, output_dim, name):
        return tf.get_variable(name,shape=[output_dim])

    def __init__(self, input_dim, output_dim, hidden_dim, fc_dim,train):

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        # [batch size x seq length x input dim]
        self.input = tf.placeholder('float', shape=[None, None, self.input_dim])
        self.labels = tf.placeholder('float', shape=[None, output_dim])
        self.time = tf.placeholder('float', shape=[None, None])
        self.keep_prob = tf.placeholder(tf.float32)

        if train == 1:


            self.Wi = self.init_weights(self.input_dim, self.hidden_dim, name='Input_Hidden_weight',reg=None)
            self.Ui = self.init_weights(self.hidden_dim, self.hidden_dim, name='Input_State_weight',reg=None)
            self.bi = self.init_bias(self.hidden_dim, name='Input_Hidden_bias')

            self.Wf = self.init_weights(self.input_dim, self.hidden_dim, name='Forget_Hidden_weight',reg=None)
            self.Uf = self.init_weights(self.hidden_dim, self.hidden_dim, name='Forget_State_weight',reg=None)
            self.bf = self.init_bias(self.hidden_dim, name='Forget_Hidden_bias')

            self.Wog = self.init_weights(self.input_dim, self.hidden_dim, name='Output_Hidden_weight',reg=None)
            self.Uog = self.init_weights(self.hidden_dim, self.hidden_dim, name='Output_State_weight',reg=None)
            self.bog = self.init_bias(self.hidden_dim, name='Output_Hidden_bias')

            self.Wc = self.init_weights(self.input_dim, self.hidden_dim, name='Cell_Hidden_weight',reg=None)
            self.Uc = self.init_weights(self.hidden_dim, self.hidden_dim, name='Cell_State_weight',reg=None)
            self.bc = self.init_bias(self.hidden_dim, name='Cell_Hidden_bias')

            self.W_decomp = self.init_weights(self.hidden_dim, self.hidden_dim, name='Decomposition_Hidden_weight',reg=None)
            self.b_decomp = self.init_bias(self.hidden_dim, name='Decomposition_Hidden_bias_enc')

            self.Wo = self.init_weights(self.hidden_dim, fc_dim, name='Fc_Layer_weight',reg=None)#tf.contrib.layers.l2_regularizer(scale=0.001)
            self.bo = self.init_bias(fc_dim, name='Fc_Layer_bias')

            self.W_softmax = self.init_weights(fc_dim, output_dim, name='Output_Layer_weight',
                                               reg=None)#tf.contrib.layers.l2_regularizer(scale=0.001)
            self.b_softmax = self.init_bias(output_dim, name='Output_Layer_bias')

        else:


            self.Wi = self.no_init_weights(self.input_dim, self.hidden_dim, name='Input_Hidden_weight')
            self.Ui = self.no_init_weights(self.hidden_dim, self.hidden_dim, name='Input_State_weight')
            self.bi = self.no_init_bias(self.hidden_dim, name='Input_Hidden_bias')

            self.Wf = self.no_init_weights(self.input_dim, self.hidden_dim, name='Forget_Hidden_weight')
            self.Uf = self.no_init_weights(self.hidden_dim, self.hidden_dim, name='Forget_State_weight')
            self.bf = self.no_init_bias(self.hidden_dim, name='Forget_Hidden_bias')

            self.Wog = self.no_init_weights(self.input_dim, self.hidden_dim, name='Output_Hidden_weight')
            self.Uog = self.no_init_weights(self.hidden_dim, self.hidden_dim, name='Output_State_weight')
            self.bog = self.no_init_bias(self.hidden_dim, name='Output_Hidden_bias')

            self.Wc = self.no_init_weights(self.input_dim, self.hidden_dim, name='Cell_Hidden_weight')
            self.Uc = self.no_init_weights(self.hidden_dim, self.hidden_dim, name='Cell_State_weight')
            self.bc = self.no_init_bias(self.hidden_dim, name='Cell_Hidden_bias')

            self.W_decomp = self.no_init_weights(self.hidden_dim, self.hidden_dim, name='Decomposition_Hidden_weight')
            self.b_decomp = self.no_init_bias(self.hidden_dim, name='Decomposition_Hidden_bias_enc')

            self.Wo = self.no_init_weights(self.hidden_dim, fc_dim, name='Fc_Layer_weight')
            self.bo = self.no_init_bias(fc_dim, name='Fc_Layer_bias')

            self.W_softmax = self.no_init_weights(fc_dim, output_dim, name='Output_Layer_weight')
            self.b_softmax = self.no_init_bias(output_dim, name='Output_Layer_bias')





    def TLSTM_Unit(self, prev_hidden_memory, concat_input):
        prev_hidden_state, prev_cell = tf.unstack(prev_hidden_memory)

        batch_size = tf.shape(concat_input)[0]
        x = tf.slice(concat_input, [0,1], [batch_size, self.input_dim])
        t = tf.slice(concat_input, [0,0], [batch_size,1])

        # Dealing with time irregularity

        # Map elapse time in days or months
        T = self.map_elapse_time(t)

        # Decompose the previous cell if there is a elapse time
        C_ST = tf.nn.tanh(tf.matmul(prev_cell, self.W_decomp) + self.b_decomp)
        C_ST_dis = tf.multiply(T, C_ST)
        # if T is 0, then the weight is one
        prev_cell = prev_cell - C_ST + C_ST_dis


        # Input gate
        i = tf.sigmoid(tf.matmul(x, self.Wi) + tf.matmul(prev_hidden_state, self.Ui) + self.bi)

        # Forget Gate
        f = tf.sigmoid(tf.matmul(x, self.Wf) + tf.matmul(prev_hidden_state, self.Uf) + self.bf)


        # Output Gate
        o = tf.sigmoid(tf.matmul(x, self.Wog) + tf.matmul(prev_hidden_state, self.Uog) + self.bog)

        # Candidate Memory Cell
        C = tf.nn.tanh(tf.matmul(x, self.Wc) + tf.matmul(prev_hidden_state, self.Uc) + self.bc)

        # Current Memory cell
        Ct = f * prev_cell + i * C

        # Current Hidden state
        current_hidden_state = o * tf.nn.tanh(Ct)

        return tf.stack([current_hidden_state, Ct])

    def get_states(self): # Returns all hidden states for the samples in a batch
        batch_size = tf.shape(self.input)[0]
        scan_input_ = tf.transpose(self.input, perm=[2, 0, 1])
        scan_input = tf.transpose(scan_input_) #scan input is [seq_length x batch_size x input_dim]
        scan_time = tf.transpose(self.time) # scan_time [seq_length x batch_size]
        initial_hidden = tf.zeros([batch_size, self.hidden_dim], tf.float32)
        ini_state_cell = tf.stack([initial_hidden, initial_hidden])
        # make scan_time [seq_length x batch_size x 1]
        scan_time = tf.reshape(scan_time, [tf.shape(scan_time)[0],tf.shape(scan_time)[1],1])
        concat_input = tf.concat([scan_time, scan_input],2) # [seq_length x batch_size x input_dim+1]
        packed_hidden_states = tf.scan(self.TLSTM_Unit, concat_input, initializer=ini_state_cell, name='states')
        all_states = packed_hidden_states[:, 0, :, :]
        return all_states


    def get_output(self, state):
        output = tf.nn.relu(tf.matmul(state, self.Wo) + self.bo)
        output = tf.nn.dropout(output, self.keep_prob)
        output = tf.matmul(output, self.W_softmax) + self.b_softmax
        return output

    def get_outputs(self): # Returns all the outputs
        all_states = self.get_states()
        all_outputs = tf.map_fn(self.get_output, all_states)
        output = tf.reverse(all_outputs, [0])[0, :, :] # Compatible with tensorflow 1.2.1
        # output = tf.reverse(all_outputs, [True, False, False])[0, :, :] # Compatible with tensorflow 0.12.1
        return output

    def get_cost_acc(self):
        logits = self.get_outputs()
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.labels, logits=logits))
        y_pred = tf.argmax(logits, 1)
        y = tf.argmax(self.labels, 1)
        return cross_entropy, y_pred, y, logits, self.labels


    def map_elapse_time(self, t):

        c1 = tf.constant(1, dtype=tf.float32)
        c2 = tf.constant(2.7183, dtype=tf.float32)

        # T = tf.multiply(self.wt, t) + self.bt

        T = tf.div(c1, tf.log(t + c2), name='Log_elapse_time')

        Ones = tf.ones([1, self.hidden_dim], dtype=tf.float32)

        T = tf.matmul(T, Ones)

        return T



We import the packages that we are going to use

In [46]:
import numpy as np
import scipy.io as sio
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
import sys
import math


In [47]:
# First we concatenate all the dataframes, the ones with class = 0 with the others, with class = 1
ap_class   = pd.concat([ap_class0,ap_class1_copies],    axis=0)
diag_class = pd.concat([diag_class0,diag_class1_copies],axis=0)
elp_class  = pd.concat([elp_class0,elp_class1_copies],  axis=0)
ei_class   = pd.concat([ei_class0,ei_class1_copies],    axis=0)
pdf_class  = pd.concat([pdf_class0,pdf_class1_copies],  axis=0)
pdnf_class = pd.concat([pdnf_class0,pdnf_class1_copies],axis=0)
pnd_class  = pd.concat([pnd_class0,pnd_class1_copies],  axis=0)

In [48]:
ap_class.sort_values(by=['idcentro','idana']).head(5)

,idcentro,idana,idcopy,sesso,annodiagnosidiabete,tipodiabete,annonascita,annoprimoaccesso,annodecesso,label
0,1,5,1,M,1986.0,5,1942,1990.0,2014.0,1
1,1,5,2,M,1986.0,5,1942,1990.0,2014.0,1
2,1,5,3,M,1986.0,5,1942,1990.0,2014.0,1
3,1,46,1,F,1982.0,5,1937,1997.0,2016.0,1
4,1,46,2,F,1982.0,5,1937,1997.0,2016.0,1


In [49]:
# At first, we merge with the patient data
features_Tlstm = pd.merge(diag_class, ap_class, on=['idcentro','idana','idcopy'])

# We create a single ID column that combines the other three:
features_Tlstm['id'] = features_Tlstm.apply(lambda x: f"{x['idcentro']}_{x['idana']}_{x['idcopy']}",axis=1)

# We reorder the columns
features_Tlstm = features_Tlstm[['id']+[x for x in features_Tlstm.columns if x!='id']]

# We drop the other ID_columns
features_Tlstm.drop(columns=['idcentro','idana','idcopy'], inplace=True)

# We categorize the columns that contain text
categorical_columns = ['codiceamd', 'valore', 'sesso']
for col in categorical_columns:
    features_Tlstm[col] = features_Tlstm[col].astype('category')
    features_Tlstm[col] = features_Tlstm[col].cat.codes

# We convert every columns into float type
numerical_columns = [col for col in features_Tlstm.columns if col not in ['id','data']]
for col in numerical_columns:
    features_Tlstm[col] = features_Tlstm[col].astype('float')
features_Tlstm['data']=features_Tlstm['data'].replace({'-':''}, regex=True)
features_Tlstm.head(10)


<ipython-input-49-ac2f9bedcdcd>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_Tlstm.drop(columns=['idcentro','idana','idcopy'], inplace=True)
<ipython-input-49-ac2f9bedcdcd>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_Tlstm[col] = features_Tlstm[col].astype('category')
<ipython-input-49-ac2f9bedcdcd>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

,id,data,codiceamd,valore,sesso,annodiagnosidiabete,tipodiabete,annonascita,annoprimoaccesso,annodecesso,label
0,1_60_1,20090716,46.0,550.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0
1,1_60_1,20090429,6.0,253.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0
2,1_60_1,20090429,43.0,267.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0
3,1_60_1,20081203,32.0,550.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0
4,1_60_1,20081203,43.0,196.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0
5,1_60_1,20081203,43.0,193.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0
6,1_60_1,20081110,46.0,550.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0
7,1_60_1,20070729,46.0,550.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0
8,1_60_1,20070202,7.0,170.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0
9,1_60_1,20070202,43.0,267.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0


In [50]:
#We define the columns that we are going to use for the labels_train, data_train and elapsed_train.
X_columns = [col for col in features_Tlstm if col not in ['id','label','data']]
T_columns = [col for col in features_Tlstm if col in ['data']]
y_columns = ['label']

Now we create variable-size batches such that each batch contains only the events associated to the previous patients each step:

In [51]:
grouped_events = features_Tlstm.groupby(['id'])
i=1
data_train_batches=list()
labels_train_batches=list()
elapsed_train_batches=list()
number_train_batches=0
for it, (ids, features) in enumerate(grouped_events):
    batch = features[features['id']==ids].sort_values(['data'])
    X = batch[X_columns]
    X=np.resize(X, (X.shape[0],1,X.shape[1]))
    Z = batch [T_columns]
    data_train_batches.append(X)
    labels_train_batches.append(batch[y_columns])
    elapsed_train_batches.append(Z)
    number_train_batches+=1
    if it % 200 == 0:
        print(f"Patient {it}/{len(ap_class)}")


<ipython-input-51-7e4af6032a6a>:7: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for it, (ids, features) in enumerate(grouped_events):


Patient 0/4284
Patient 200/4284
Patient 400/4284
Patient 600/4284
Patient 800/4284
Patient 1000/4284
Patient 1200/4284
Patient 1400/4284
Patient 1600/4284
Patient 1800/4284
Patient 2000/4284
Patient 2200/4284
Patient 2400/4284
Patient 2600/4284
Patient 2800/4284
Patient 3000/4284
Patient 3200/4284
Patient 3400/4284
Patient 3600/4284
Patient 3800/4284
Patient 4000/4284


In [52]:
def training(learning_rate,training_epochs,train_dropout_prob,hidden_dim,fc_dim):

    input_dim = data_train_batches[0].shape[2]
    output_dim = labels_train_batches[0].shape[1]

    lstm = TLSTM(input_dim, output_dim, hidden_dim, fc_dim,1)

    cross_entropy, y_pred, y, logits, labels = lstm.get_cost_acc()
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(training_epochs):  #
            # Loop over all batches
            total_cost = 0
            for i in range(number_train_batches):  #
                # batch_xs is [number of patients x sequence length x input dimensionality]
                batch_xs, batch_ys, batch_ts = data_train_batches[i], labels_train_batches[i], \
                                                         elapsed_train_batches[i]
                batch_ts = np.reshape(batch_ts, [batch_ts.shape[0], batch_ts.shape[1]])
                sess.run(optimizer,feed_dict={lstm.input: batch_xs, lstm.labels: batch_ys,\
                                              lstm.keep_prob:train_dropout_prob, lstm.time:batch_ts})


        print("Training is over!")


        Y_pred = []
        Y_true = []
        Labels = []
        Logits = []
        for i in range(number_train_batches):
            batch_xs, batch_ys, batch_ts = data_train_batches[i], labels_train_batches[i], \
                                                     elapsed_train_batches[i]
            batch_ts = np.reshape(batch_ts, [batch_ts.shape[0], batch_ts.shape[1]])
            c_train, y_pred_train, y_train, logits_train, labels_train = sess.run(lstm.get_cost_acc(), feed_dict={
                lstm.input:
                                                                                                       batch_xs, lstm.labels: batch_ys, \
                                           lstm.keep_prob: train_dropout_prob, lstm.time: batch_ts})

            if i > 0:
                Y_true = np.concatenate([Y_true, y_train], 0)
                Y_pred = np.concatenate([Y_pred, y_pred_train], 0)
                Labels = np.concatenate([Labels, labels_train], 0)
                Logits = np.concatenate([Logits, logits_train], 0)
            else:
                Y_true = y_train
                Y_pred = y_pred_train
                Labels = labels_train
                Logits = logits_train

        total_acc = accuracy_score(Y_true, Y_pred)
        total_auc = roc_auc_score(Labels, Logits, average='micro')
        total_auc_macro = roc_auc_score(Labels, Logits, average='macro')
        print("Train Accuracy = {:.3f}".format(total_acc))
        print("Train AUC = {:.3f}".format(total_auc))
        print("Train AUC Macro = {:.3f}".format(total_auc_macro))


Now we train our model

In [53]:
learning_rate = 0.01
training_epochs = 50
dropout_prob = 0.5
hidden_dim = 1
fc_dim = 1
training(learning_rate, training_epochs, dropout_prob, hidden_dim, fc_dim)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



KeyboardInterrupt: ignored

### Exercise 2.3 - *PubMedBERT*

We read the *.csv* that contain all the information to translate *codiceamd* and *codiceatc* into descriptive texts

In [54]:
read_data_path  = "/content/drive/MyDrive/ML/codici"

codiciAMD = pd.read_csv(read_data_path+'/amd_codes_for_bert.csv', header=0 ,names=['codiceamd','meaning'])
codiciATC = pd.read_csv(read_data_path+'/atc_info_nodup.csv', header=0 ,names=['codiceatc','first','last','len','numpatients','numdates','atc_nome'])
codiciATC = codiciATC.filter(['codiceatc','atc_nome']).rename(columns={'atc_nome':'meaning'})

In [55]:
codiciAMD.head()

,codiceamd,meaning
0,AMD090,Diet only
1,AMD140,Self control
2,AMD215,Number of strips prescribed per week
3,AMD228,Integrated management
4,AMD086,Self-monitoring of blood glucose


In [56]:
codiciATC.head()

,codiceatc,meaning
0,A10AB01,insulin (human)
1,A10AB04,insulin lispro
2,A10AB05,insulin aspart
3,A10AB06,insulin glulisine
4,A10AC01,insulin (human)


We add the *comment-text* column:

In [57]:
def addingMeaningColumn(df:pd.DataFrame, patients:pd.DataFrame) -> pd.DataFrame:
    initialCols = df.columns.to_list()
    # Do we have to translate AMD or ATC codes?
    if np.isin('codiceamd',list(df.columns)):
        codice = 'codiceamd'
        codiciDF = codiciAMD
    else:
        codice = 'codiceatc'
        codiciDF = codiciATC

    # At first, we add a column into the dataframe with the descriptive text of each code
    df = pd.merge(df, codiciDF, on=[codice])

    # Now we add the information about the patient
    df = pd.merge(df, patients, on=['idcentro','idana','idcopy']).sort_values(by='data')

    # Rename 'meaning' column to 'comment_text'
    df = df.rename(columns={'meaning':'comment_text'})

    initialCols.remove(codice)
    initialCols.remove('data')
    initialCols.append('comment_text')
    return df.filter(items=initialCols)

In [58]:
diag_class_bert = addingMeaningColumn(diag_class, ap_class)
elp_class_bert  = addingMeaningColumn(elp_class, ap_class)
ei_class_bert   = addingMeaningColumn(ei_class, ap_class)
pdf_class_bert  = addingMeaningColumn(pdf_class, ap_class)
pdnf_class_bert = addingMeaningColumn(pdnf_class, ap_class)
pnd_class_bert  = addingMeaningColumn(pnd_class, ap_class)

Let's eliminate NaN values:

In [59]:
diag_class_bert = diag_class_bert.fillna("")
elp_class_bert  = elp_class_bert.fillna("")
ei_class_bert   = ei_class_bert.fillna("")
pdf_class_bert  = pdf_class_bert.fillna("")
pdnf_class_bert = pdnf_class_bert.fillna("")
pnd_class_bert  = pnd_class_bert.fillna("")

In [60]:
diag_class_bert.head()

,idcentro,idana,idcopy,valore,comment_text
9532,74,7564,1,N,Cigarette smoke
85067,75,405,2,N,Cigarette smoke
85153,75,405,3,N,Cigarette smoke
119254,157,3411,1,EX,Cigarette smoke
137937,118,6285,2,EX,Cigarette smoke


In [61]:
ap_class = ap_class.reset_index()

Now we create a paragraph using all the events of all tables for each patient in order to use a BERT-like model:

In [62]:
for i in range(len(ap_class)):

    patient = dict(ap_class.loc[i])
    patientDF = pd.DataFrame(data=patient, index=[i])

    patient_text = f"I was born in {patient['annonascita']} and I am a {'man' if patient['sesso']=='M' else 'woman'}. I was diagnosed in {patient['annodiagnosidiabete']} with diabetes of type {patient['tipodiabete']}. My first access was in {patient['annoprimoaccesso']}."

    # We aggregate the events to the text.
    pdnf_class_bert_patient = pdnf_class_bert.merge(patientDF, on=['idcentro','idcopy','idana'])
    patient_text += " My diets and blood glucose controls have been: "
    for index, event in pdnf_class_bert_patient.iterrows():
        patient_text += f"{event.comment_text} with value {event.valore}, "
    patient_text = patient_text[:-2] + "."

    pdf_class_bert_patient  = pdf_class_bert.merge(patientDF, on=['idcentro','idcopy','idana'])
    patient_text += " The diabetes drugs that have been prescripted to me: "
    for index, event in pdf_class_bert_patient.iterrows():
        patient_text += f"{event.comment_text} with quantity {event.quantita}, "
    patient_text = patient_text[:-2] + "."

    pnd_class_bert_patient  = pnd_class_bert.merge(patientDF, on=['idcentro','idcopy','idana'])
    patient_text += " The non-diabetes drugs that have been prescripted to me: "
    for index, event in pnd_class_bert_patient.iterrows():
        patient_text += f"{event.comment_text} with value {event.valore}, "
    patient_text = patient_text[:-2] + "."

    ei_class_bert_patient   = ei_class_bert.merge(patientDF, on=['idcentro','idcopy','idana'])
    patient_text += " Medical tests: "
    for index, event in ei_class_bert_patient.iterrows():
        patient_text += f"{event.comment_text} with value {event.valore}, "
    patient_text = patient_text[:-2] + "."

    elp_class_bert_patient  = elp_class_bert.merge(patientDF, on=['idcentro','idcopy','idana'])
    patient_text += " Laboratory tests: "
    for index, event in elp_class_bert_patient.iterrows():
        patient_text += f"{event.comment_text} with value {event.valore}, "
    patient_text = patient_text[:-2] + "."

    diag_class_bert_patient = diag_class_bert.merge(patientDF, on=['idcentro','idcopy','idana'])
    patient_text += " Diagnosis tests: "
    for index, event in diag_class_bert_patient.iterrows():
        patient_text += f"{event.comment_text} with value {event.valore}, "
    patient_text = patient_text[:-2] + "."

    patient_text += f" Finally i died in {patient['annodecesso']}."
    ap_class.loc[i, 'text'] = patient_text

In [63]:
ap_class_bert = ap_class.filter(items=['text','label'])

In [64]:
ap_class_bert.head()

,text,label
0,I was born in 1923 and I am a woman. I was dia...,0
1,I was born in 1930 and I am a man. I was diagn...,0
2,I was born in 1934 and I am a man. I was diagn...,0
3,I was born in 1936 and I am a woman. I was dia...,0
4,I was born in 1923 and I am a woman. I was dia...,0


Now let's apply the PubMedBERT:

In [67]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.5 MB/s eta 0:00:00


In [68]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

model = AutoModelForMaskedLM.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

model = model.to(DEVICE)

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


We define some model HyperParameters:

In [69]:
BATCH_SIZE = 64
LEARNING_RATE = 1e-5
EPOCHS = 4

In [70]:
# Let's use a maximum length of 200.
max_len = 200

In [71]:
# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []
attention_masks = []

for i in range(len(ap_class_bert)):
    paragraph = ap_class_bert.loc[i,'text']
    encoded_dict = tokenizer.encode_plus(paragraph,
                                         add_special_tokens=True,
                                         max_length=max_len,
                                         truncation=True,
                                         padding='max_length',
                                         return_attention_mask=True,
                                         return_tensors='pt')
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids=torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(ap_class_bert['label'].to_list())

print('DONE.')

DONE.


Now that it’s done, we can divide up the samples into the three splits.

In [72]:
train_size = int(0.8 * len(ap_class_bert))
val_size = int(0.1 * len(ap_class_bert))
test_size = len(ap_class_bert) - (train_size + val_size)

In [73]:
indexes = np.arange(0, len(ap_class_bert))
random.shuffle(indexes)

In [74]:
train_idx = indexes[0:train_size]
val_idx = indexes[train_size:(train_size + val_size)]
test_idx = indexes[(train_size + val_size):]

In [75]:
print(f"Training Size:   {train_size}")
print(f"Validation Size: {val_size}")
print(f"Test Size:       {test_size}")

Training Size:   3427
Validation Size: 428
Test Size:       429


In [76]:
from torch.utils.data import TensorDataset

# Split the samples, and create TensorDatasets for each split.
train_dataset = TensorDataset(input_ids[train_idx], attention_masks[train_idx], labels[train_idx])
val_dataset = TensorDataset(input_ids[val_idx], attention_masks[val_idx], labels[val_idx])
test_dataset = TensorDataset(input_ids[test_idx], attention_masks[test_idx], labels[test_idx])

Prepare the Data Loader:

In [77]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order.
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = BATCH_SIZE # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = BATCH_SIZE # Evaluate with this batch size.
)

In [78]:
from torch.optim import AdamW

# Note: AdamW is a class from the huggingface library (as opposed to pytorch)
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = LEARNING_RATE,
                  eps = 1e-8
                )

We also implemet a LR scheduler so as to implement learning rate decay:

In [79]:
from transformers import get_linear_schedule_with_warmup

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples!)
total_steps = len(train_dataloader) * EPOCHS

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [80]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [81]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [82]:
import time

# We'll store a number of quantities such as training and validation loss,
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, EPOCHS):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print(f'======== Epoch {epoch_i} / {EPOCHS} ========')
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print(f'  Batch {step:>5,}  of  {len(train_dataloader):>5,}.    Elapsed: {elapsed}.')

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(DEVICE)
        b_input_mask = batch[1].to(DEVICE)
        b_labels = batch[2].to(DEVICE)

        # Always clear any previously calculated gradients before performing a backward pass
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        result = model(b_input_ids,
                       token_type_ids=None,
                       attention_mask=b_input_mask,
                       labels=b_labels,
                       return_dict=True)

        loss = result.loss
        logits = result.logits

        # Accumulate the training loss over all of the batches so that we can calculate the average loss at the end.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print(f"  Average training loss: {avg_train_loss:.2f}")
    print(f"  Training epcoh took: {training_time}")

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(DEVICE)
        b_input_mask = batch[1].to(DEVICE)
        b_labels = batch[2].to(DEVICE)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            result = model(b_input_ids,
                           token_type_ids=None,
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        # Get the loss and "logits" output by the model. The "logits" are the
        # output values prior to applying an activation function like the
        # softmax.
        loss = result.loss
        logits = result.logits

        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)


    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print(f"  Accuracy: {avg_val_accuracy:.2f}")

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {avg_val_loss:.2f}")
    print("  Validation took: {validation_time}")

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print(f"Total training took {format_time(time.time()-total_t0)} (h:mm:ss)")


======== Epoch 0 / 4 ========
Training...


KeyboardInterrupt: ignored